In [1]:
from DICOMLib import DICOMUtils
import vtkSegmentationCorePython as vtkSegmentationCore
import numpy


# Code from https://github.com/SlicerRt/SlicerRT/blob/master/BatchProcessing/BatchStructureSetConversion.py
def ConvertStructureSetToLabelmap():
    # ConvertStructureSetToLabelmap()
    labelmapsToSave = []

    # Get all segmentation nodes from the scene
    segmentationNodes = slicer.util.getNodes('vtkMRMLSegmentationNode*')

    for segmentationNode in segmentationNodes.values():
        # Set referenced volume as rasterization reference
        referenceVolume = slicer.vtkSlicerDicomRtImportExportModuleLogic.GetReferencedVolumeByDicomForSegmentation(
            segmentationNode)
        if referenceVolume == None:
            continue

    # Perform conversion
    binaryLabelmapRepresentationName = vtkSegmentationCore.vtkSegmentationConverter.GetSegmentationBinaryLabelmapRepresentationName()
    segmentation = segmentationNode.GetSegmentation()
    segmentation.CreateRepresentation(binaryLabelmapRepresentationName)

    # Create labelmap volume nodes from binary labelmaps
    segmentIDs = vtk.vtkStringArray()
    segmentation.GetSegmentIDs(segmentIDs)
    for segmentIndex in range(0, segmentIDs.GetNumberOfValues()):
        segmentID = segmentIDs.GetValue(segmentIndex)
        segment = segmentation.GetSegment(segmentID)
        binaryLabelmap = segment.GetRepresentation(
            vtkSegmentationCore.vtkSegmentationConverter.GetSegmentationBinaryLabelmapRepresentationName())
        if not binaryLabelmap:
            continue
        labelmapNode = slicer.vtkMRMLLabelMapVolumeNode()
        slicer.mrmlScene.AddNode(labelmapNode)
        labelmapName = segmentationNode.GetName() + "_" + segmentID
        labelmapNode.SetName(labelmapName)
        if not slicer.vtkSlicerSegmentationsModuleLogic.CreateLabelmapVolumeFromOrientedImageData(
          binaryLabelmap, labelmapNode):
            continue

        # Append volume to list
        labelmapsToSave.append(labelmapNode)# SaveLabelmaps into output_dir

    return labelmapsToSave


def SaveLabelmaps(labelmapsToSave, outputDir):
    for labelmapNode in labelmapsToSave:
        # Clean up file name and set path
        fileName = labelmapNode.GetName() + '.nrrd'
        table = str.maketrans(dict.fromkeys('!?:;'))
        fileName = fileName.translate(table)
        filePath = outputDir + '/' + fileName
        logging.info('  Saving structure ' + labelmapNode.GetName() + '\n    to file ' + fileName)

    # Save to file
    success = slicer.util.saveNode(labelmapNode, filePath)
    if not success:
        logging.error('Failed to save labelmap: ' + filePath)


# Exporting data from all testing/training brachytherapy patients
folder_prefix = 'Brachy patients/PYPB'


for patient_num in range(1, 31):
    # Clear scene
    slicer.mrmlScene.Clear(False)

    #concatenate folder name
    folder_name = folder_prefix + str(patient_num).zfill(2)
    
    # Load dicom folder
    DICOMUtils.openTemporaryDatabase()
    DICOMUtils.importDicom(folder_name)

    #Load first patient into slicer
    patient = slicer.dicomDatabase.patients()[0]
    DICOMUtils.loadPatientByUID(patient)

    output_dir = slicer.app.temporaryPath + '/BrachyPatients/Output/' + str(patient_num).zfill(2)
    labelmaps = ConvertStructureSetToLabelmap()
    SaveLabelmaps(labelmaps, output_dir)
    
    # Get first plan node
    planNodes = slicer.util.getNodes('vtkMRMLRTPlanNode*')
    planNode = list(planNodes.values())[0]
    # Get ultrasound volume node
    ultrasoundNodes = slicer.util.getNodes('vtkMRMLScalarVolumeNode*')
    ultrasoundNode = list(ultrasoundNodes.values())[0]
    
    # Get urethra representation - including urethra
    RTstruct = slicer.util.getNodes('vtkMRMLSegmentationNode1')
    RTstruct = list(RTstruct.values())[0]
    RTstruct.CreateBinaryLabelmapRepresentation()
    urethra = RTstruct.GetBinaryLabelmapRepresentation('Urethra')

    # Get catheter nodes (children of plan in SH)
    shNode = slicer.vtkMRMLSubjectHierarchyNode.GetSubjectHierarchyNode(slicer.mrmlScene)
    planItemID = shNode.GetItemByDataNode(planNode)
    children = vtk.vtkIdList()
    shNode.GetItemChildren(planItemID, children)

    catheterNodes = []
    for i in range(children.GetNumberOfIds()):
        child = children.GetId(i)
        catheterNode = shNode.GetItemDataNode(child)
        catheterNodes.append(catheterNode)

    # Convert each catheter node into labelmap nodes
    modelNodes = []
    for catheterNode in catheterNodes:
        # 1. Get model from catheter markups curve
        catheterNode.SetCurveTypeToLinear()
        polydata = catheterNode.GetCurveWorld()
        # add catheter diameter
        polydataTube = vtk.vtkTubeFilter()
        polydataTube.SetInputData(polydata)
        polydataTube.SetNumberOfSides(20)
        # 5F catheter radius
        polydataTube.SetRadius(0.7366)
        polydataTube.Update()
        # show on slicer for visualizing purposes
        modelNode = slicer.vtkMRMLModelNode()
        slicer.mrmlScene.AddNode(modelNode)
        modelNode.SetAndObservePolyData(polydataTube.GetOutput())
        # modelNode.CreateDefaultDisplayNodes()
        modelNodes.append(modelNode)

    #Create segmentation node
    segmentationNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLSegmentationNode')
    # Convert all model nodes to segments
    for modelNode in modelNodes:
        modelSegment = slicer.vtkSlicerSegmentationsModuleLogic().CreateSegmentFromModelNode(modelNode)
        segmentationNode.GetSegmentation().AddSegment(modelSegment)
    # Add urethra
    segmentationNode.AddSegmentFromBinaryLabelmapRepresentation(urethra, 'Urethra')

    # Display segmentation node 
    slicer.mrmlScene.AddNode(segmentationNode)
    segmentationNode.CreateDefaultDisplayNodes()
    
    # Create LabelMapVolumeNode for all segments
    labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    # Export multiple segments to label map volume node
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentationNode, labelmapVolumeNode, ultrasoundNode)
    
    # Export Segmentation to numpy array
    output_file = 'numpy_data/Patient Segmentations/brachy_patient' + str(patient_num).zfill(2)
    segmentationArray = arrayFromVolume(labelmapVolumeNode)
    numpy.save(output_file, segmentationArray)

    # Export ultrasound volume node to numpy array
    ultrasound_file = 'numpy_data/Ultrasound Volumes/ultrasound' + str(patient_num).zfill(2)
    ultrasoundArray = arrayFromVolume(ultrasoundNode)
    numpy.save(ultrasound_file, ultrasoundArray)

Please install SlicerRT extension to enable loading of DICOM RT Structure Set objects
Please install SlicerRT extension to enable loading of DICOM RT Structure Set objects
Irregular volume geometry detected, but maximum error non-zero but is within tolerance (maximum error of 3.63135e-06 mm, tolerance threshold is 0.001 mm).
Traceback (most recent call last):
  File "<string>", line 154, in <module>
  File "C:\Users\z.hu\AppData\Local\NA-MIC\Slicer 4.11.0-2019-06-19\lib\Python\lib\site-packages\numpy-1.16.2-py3.6-win-amd64.egg\numpy\lib\npyio.py", line 517, in save
    fid = open(file, "wb")
FileNotFoundError: [Errno 2] No such file or directory: 'downloads/numpy_data/Patient Segmentations/brachy_patient01.npy'
